# Chat via LangGraph

In [7]:
import { START, END, MessagesAnnotation, StateGraph, MemorySaver } from "@langchain/langgraph";
import { SystemMessage, HumanMessage } from "@langchain/core/messages";
import { Annotations } from "@common/annotations.ts";
import { readFileSync } from 'node:fs';

import { SERVER_DATA_DIR, SERVER_SRC_DIR } from '../server/src/util/fileUtils.ts';
import { Logger, getNotebookLogger } from '../server/src/Logger.ts';
import { newModel, responseContent } from '../server/src/agents/agent.ts';
import { makeSystemData } from '../server/src/agents/annotation.ts';

In [8]:
const logger = getNotebookLogger();
const llm = newModel("Anthropic");

// Define the function that calls the model
const modelNode = async (state: typeof MessagesAnnotation.State) => {
  const response = await llm.invoke(state.messages);
  // Update message history with response:
  return { messages: response };
};

// Define a new graph
const workflow = new StateGraph(MessagesAnnotation)
  // Define the (single) node in the graph
  .addNode("model", modelNode)
  .addEdge(START, "model")
  .addEdge("model", END);

// Add memory
const memory = new MemorySaver();
const graph = workflow.compile({ checkpointer: memory });

In [9]:
const userUUID: string = "0";
const lhsText = readFileSync(`${SERVER_DATA_DIR}/AES/selected-text.txt`, 'utf-8');
const rhsText = readFileSync(`${SERVER_DATA_DIR}/AES/pre-written.txt`, 'utf-8');
const PROMPT = readFileSync(`${SERVER_SRC_DIR}/agents/nodes/chatNodePrompt.txt`, 'utf-8');
const currentAnnotations: Annotations = { "mappings": [], "lhsLabels": [], "rhsLabels": [] };
const userInput = "Does the Lean specification define all relevant invariants and preconditions as described in the requirements?";

In [10]:
const config = { configurable: { thread_id: userUUID } };
const systemData = makeSystemData(lhsText, rhsText, currentAnnotations, logger);

logger.info(`System Message:\n${PROMPT + systemData}`);
logger.info(`Human Message:\n${userInput}`)

System Message:
You are an expert in text comparison and annotation. Your goal is to assist the user in analyzing two pieces of text and suggest annotations that capture the significant relationships between them. The text on the left (LHS TEXT) and the right (RHS TEXT) may have corresponding words, phrases, or concepts that need to be highlighted. The annotations contain mappings between corresponding phrases/words in the LHS and RHS texts, including any potential relationships or synonyms. Each annotation also comes with a description such as "Greeting," "Noun," or other suitable labels based on the context of each text, and a status such as:
   - "default" for mappings that correctly preserve concepts.
   - "warning" for possibly ambiguous or concerning mappings.
   - "error" for mappings that are incorrect or mismatched.

Example of the input texts:
```
### LHS TEXT

Hello ugly world! How are you?

### RHS TEXT

Hi beautiful planet!

### EXISTING ANNOTATIONS

[{
  description: "Gre

In [11]:
logger.info("Sending prompt to LLM.");
const output = await graph.invoke({ messages: [
  new SystemMessage(PROMPT + systemData),
  new HumanMessage(userInput)
]}, config);
logger.info("Received response from LLM.");
logger.info(responseContent(output));

Sending prompt to LLM.
Received response from LLM.
# Analysis of Invariants and Preconditions in the Lean Specification

After comparing the natural language requirements (FIPS 197 AES specification) with the Lean implementation, I can identify several areas where invariants and preconditions are either present or missing.

## Missing Invariants and Preconditions

1. **Input Size Validation**: While the Lean code has some size checks (e.g., in `shiftRows` and `mixColumns`), there are no explicit preconditions for the main AES functions that verify:
   - Input block is exactly 16 bytes (128 bits)
   - Key sizes are exactly 16, 24, or 32 bytes for AES-128, AES-192, and AES-256 respectively

2. **State Representation**: The FIPS document describes the state as a 4×4 array of bytes, but the Lean implementation uses a flat array of 16 bytes. While functionally equivalent, the code lacks explicit invariants that maintain the conceptual 4×4 structure.

3. **Key Schedule Validation**: The key 